# WechatSogou

### algorithm:
1. import module
2. get gzh historical articles
3. save the html to local disk
4. insert into database or update sids

## 1. import modules

In [14]:
import wechatsogou
import random
import requests
import datetime
import time
from lxml.etree import XMLSyntaxError
from requests.exceptions import ConnectionError
from pyquery import PyQuery as pq
import os
import pathlib
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
# 直连
ws_api = wechatsogou.WechatSogouAPI()

# 验证码输入错误的重试次数，默认为1
ws_api = wechatsogou.WechatSogouAPI(captcha_break_time=3)

# 所有requests库的参数都能在这用
# 如 配置代理，代理列表中至少需包含1个 HTTPS 协议的代理, 并确保代理可用
ws_api = wechatsogou.WechatSogouAPI(proxies={
    "http": "127.0.0.1:8888",
    "https": "127.0.0.1:8888",
})

# 如 设置超时
ws_api = wechatsogou.WechatSogouAPI(timeout=1)


In [3]:
headers = {
    'Cookie': """SUID=2F2E40613921940A000000005C7C8A68; SUV=0054C7AB61402E2F5C7C8A6C4E9DE485; IPLOC=CN3100; UM_distinctid=16af8a3d10b1c6-00075670c610bf-e353165-1fa400-16af8a3d10c66e; ABTEST=5|1560408316|v1; JSESSIONID=aaafTsCy_KK2le_X5lqRw; PHPSESSID=38sddn82q3fr3mihpq5s9j09j7; SNUID=A3091D2C494FC612B1C5C56E4A2CA7BE; sct=7""",
    'Host': 'weixin.sogou.com',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'
}

In [4]:
ua = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]


## get proxy tests

In [115]:
with open('ip.txt') as fp:
    proxy_pool = fp.readlines()


In [116]:
ls_proxy_pool = []
for string in proxy_pool:
    ip = string.strip().split("\t")
    if ip[2] == 'HTTP':
        proxy_host = ip[0]+":"+ip[1]
        ls_proxy_pool.append(proxy_host)

In [117]:
def get_proxy():
    http_url = "http://ip111.cn"
    proxy_temp = random.choice(ls_proxy_pool)
    try:
        response = requests.get(http_url)
        if response.status_code == 200:
            return proxy_temp
        return None
    except ConnectionError:
        return None


## get articles

In [ ]:
ws_api.get_gzh_info('南航青年志愿者')

In [ ]:
gzh = ws_api.search_gzh('付鹏的财经世界')

In [ ]:
list(gzh)

In [ ]:
articles = ws_api.search_article('付鹏的财经世界')

In [ ]:
list(articles)

In [ ]:
ws_api.get_sugg('财经')

### Search historical articles

In [32]:
proxy = None #将代理设为全局变量
max_count = 3 #最大请求次数

In [33]:
def get_html(url,count = 1):
    #打印一些调试信息
    print('Crawling:', url)
    print('Trying Count:', count)

    global proxy #引用全局变量
    if count >= max_count: #如果请求次数达到了上限
        print('Tried too many counts！')
        return None

    try :
        uachoice = random.choice(ua)
        headers = {'Cookie': """SUID=2F2E40613921940A000000005C7C8A68; SUV=0054C7AB61402E2F5C7C8A6C4E9DE485; IPLOC=CN3100; UM_distinctid=16af8a3d10b1c6-00075670c610bf-e353165-1fa400-16af8a3d10c66e; ABTEST=5|1560408316|v1; JSESSIONID=aaafTsCy_KK2le_X5lqRw; PHPSESSID=38sddn82q3fr3mihpq5s9j09j7; SNUID=A3091D2C494FC612B1C5C56E4A2CA7BE; sct=7""",
        "User-Agent": uachoice}
        if proxy:# 如果现在正在使用代理
            proxies = {
                'http':'http://'+ proxy #设置协议类型
            }
            response = requests.get(url, headers=headers, proxies=proxies, timeout=3) #使用有代理参数的请求
        else: #否则使用正常的请求
            response = requests.get(url, headers=headers, timeout=3)#禁止自动处理跳转
        if response.status_code == 200:
            print("请求成功！")
            return response.text
        if response.status_code == 302:
            # 需要代理
            print("302！")
            proxy = get_proxy()
            if proxy:
                print('Using Proxy', proxy)
                return get_html(url)
            else:
                print('Get Proxy Failed')
                return None
    except ConnectionError as e:
        print('Error Occurred', e.args)#打印错误信息
        proxy = get_proxy() #若失败，更换代理
        count += 1 #请求次数+1
        time.sleep(2)
        return get_html(url, count) #重试
    
    

In [92]:
# 解析详情页
def parse_detail(html):
    doc = pq(html)
    title = doc('.rich_media_title').text()
    content = doc('.rich_media_content').text()
    wechat_name = doc('#js_profile_qrcode > div > strong').text()
    nickname = doc('.rich_media_meta_text').text()
    wechat = doc('#js_profile_qrcode > div > p:nth-child(3) > span').text()
    date = None
    return {
        'title':title,
        'wechat_name': wechat_name,
        'content':content,
        'date':date,
        'nickname':nickname,
        'wechat':wechat
    }


In [118]:
with open('KOL.txt') as fp:
    ls_kol = fp.readlines()
    ls_kol = [x.replace('\n', '') for x in ls_kol]

In [119]:
ls_kol

['付鹏的财经世界',
 '长赢指数投资',
 '博策远略',
 '新全球资产配置',
 '沧海一土狗',
 '川流不息',
 'Trader.M的知乎文章',
 'StochasticWalker的知乎文章',
 '全球资产观察',
 '湖畔公寓28号',
 'Bridgewater - Daily Observations',
 '半夏投资',
 '因子投资那些事儿',
 '跨资产的因子配置',
 '宏观大类策略研究',
 'Chihiro Quantitative Research',
 '量化交易&宽客',
 '淳臻投资',
 'cfuwxd',
 '孟岩',
 '东京交易厅',
 '聚宽量化实验室',
 '饭统戴老板',
 '黑猫Q形态的知乎文章',
 '关关楼西',
 '高善文经济观察',
 'Moneycode',
 '全球经济每日漫谈',
 '月风投资笔记',
 '伦敦交易员',
 'babyquant谈量化金融',
 '期权策略交易',
 '金融学前沿论文速递',
 '李超宏观研究与资产配置',
 '人民币交易与研究',
 '筹码',
 '投研帮',
 'Pure Delta',
 '向小田',
 '【量化投资与机器学习】微信公众号',
 '量化投资大家学',
 '深圳天谷资产管理有限公司',
 '40秒',
 '岭峰资本',
 '混沌巡洋舰',
 '市场观察家',
 '外汇头条',
 '定投十年十倍',
 '私募工场',
 '聪明投资者',
 '扑克投资家',
 '看研报资讯',
 '冷眼局中人',
 '三思门',
 '华尔街情报圈',
 '冲浪团',
 '加州分析员']

loop in the list key of leaders:
get history
loop in the history article url
if there's author's directory

In [120]:
for au in tqdm(ls_kol):
    print('now: %s'%au)
    while True:
        # get latest 10 articles
        try:
            history = ws_api.get_gzh_article_by_history(au)
        except:
            pass
            print('failed to fetch historical articles.')
        else:
            break
        time.sleep(2)
        
    # get articles 
    if len(history) == 0:
        continue
    # make directory
    wechat_name = history['gzh']['wechat_name']
    export_dir = 'data/{0}/'.format(wechat_name)
    if not os.path.exists(export_dir):
        pathlib.Path(export_dir).mkdir(parents=True, exist_ok=True)
    for ci in history['article']:
        author = ci['author']
        title = ci['title']
        dt = ci['datetime']
        dt = datetime.datetime.fromtimestamp(
            int(dt)
        ).strftime('%Y_%m_%d')
        url = ci['content_url']
        print(title)
        # use requests
        title = title.replace('/', '')
        fpath = 'data/{0}/{1}_{2}_{3}.html'.format(wechat_name, dt, author, title)
        if not os.path.exists(fpath):
            article = get_html(url)
            if article:
                # export article
                with open(fpath, 'w') as fp:
                    fp.write(article)
                
        else:
            continue
        time.sleep(2)    
    time.sleep(2)




  0%|          | 0/57 [00:00<?, ?it/s]

now: 付鹏的财经世界
now: 长赢指数投资
please input code: e24kc4
2019年6月ETF计划（二）：买入两份
2019年6月ETF计划（一）：买入两份
波段策略.网格之三：网格策略进阶/2.0版
2019年5月ETF计划（二）：买入两份；卖出一份
波段策略.网格之二：网格策略基础/1.0版
波段策略.网格之一：写在前面、体系以及策略
2019年5月ETF计划（一）：买入一份
2019年4月ETF计划（一）：买入两份
围炉夜话（5）：你的肩膀可以扛起多少信任？
2019年3月ETF计划（一）：150不交易，S买入一份





  4%|▎         | 2/57 [00:15<07:19,  7.99s/it]

now: 博策远略
中金公司：定投该怎么投？—博览研报010
模型更新 190623
模型更新 190616
美股历史上都是为什么“挂”的？美联储、基本面、高估值、外部冲击—博览研报009
除二创以外均进入到布局区域——模型更新 190609
2019年A股市场招生全国统一考试试题 答对14道方可入市交（收）易（割）
投资决策中的“统计意义”与“经济含义”——从美债收曲倒挂案例说起
模型更新 190602
[视频]全球TOP10上市公司市值变迁,2007年的中石油亮了
模型更新 190526
全市场最贵消费指数的未来
模型更新 190519
A股：“消了户你也不会幸福的！”
北上资金到底赚了多少钱？跟着抄作业的正确姿势？——博览研报008
模型更新 191512
171倍市盈率的创业板综告诉你什么叫价值陷阱
模型更新 190428





  5%|▌         | 3/57 [00:18<05:43,  6.36s/it]

...




 96%|█████████▋| 55/57 [11:18<01:19, 39.73s/it]

now: 冲浪团
本周市场观点
Crawling: http://mp.weixin.qq.com/s?timestamp=1561865988&src=3&ver=1&signature=xNq6M1LNqUqHSjEY9j6kFBrR1SM-3hcTpb9YabEmeu1Cf37noybpIulCrki353NDy3a-MyxmXZfGABcbBFL0rn8x4*LUucfGCvxlGbRqB8mY8KMpxtKqhOOQ8e1PzPqlIaomN3T4876PEw8q-jTgo2D3RPc-C9mTtNHvCQoOUN4=
Trying Count: 1
请求成功！
谨慎操作；但是不看空，看多
Crawling: http://mp.weixin.qq.com/s?timestamp=1561865988&src=3&ver=1&signature=xNq6M1LNqUqHSjEY9j6kFBrR1SM-3hcTpb9YabEmeu1Cf37noybpIulCrki353NDy3a-MyxmXZfGABcbBFL0rkpQXFd4z4q3reC9IAMqQtrVD-Uyt4Y8cdwBD5bs0Dk8GgYUF-GQ2q9gyTGF*9atWi4eEqdcUKcG3qXZLRVnPzs=
Trying Count: 1
请求成功！
观点不改
Crawling: http://mp.weixin.qq.com/s?timestamp=1561865988&src=3&ver=1&signature=xNq6M1LNqUqHSjEY9j6kFBrR1SM-3hcTpb9YabEmeu1Cf37noybpIulCrki353NDy3a-MyxmXZfGABcbBFL0rvR4DFhrKi*zdnpnbFrsdcQ9kFRiohDvaRm2b1Mf5y7uYJvg6NYp8VuUBbx51Hm5qwlUL9EUdmK*J2jfRlU6sEg=
Trying Count: 1
请求成功！
震荡上行
Crawling: http://mp.weixin.qq.com/s?timestamp=1561865988&src=3&ver=1&signature=xNq6M1LNqUqHSjEY9j6kFBrR1SM-3hcTpb9YabEmeu1Cf37noybpIulCrki




 98%|█████████▊| 56/57 [11:42<00:35, 35.08s/it]

now: 加州分析员
美股是否已经走到尽头？
Crawling: http://mp.weixin.qq.com/s?timestamp=1561866012&src=3&ver=1&signature=xNq6M1LNqUqHSjEY9j6kFKgsk9J5arq-dN9C9hzIeycyuRwNaC9sgrOWWIJBkvGeldBrf9Iwcj7CvqlMauwrXCiEi-*yr9WDVXuWf6b5li911a-PkfKpoAtysAK9pB6mQPzD0ZM6fQTaDr5yR*2Jw2Z7efFm1FESvFOKa3-2pHY=
Trying Count: 1
请求成功！
新债王3月电话会
Crawling: http://mp.weixin.qq.com/s?timestamp=1561866012&src=3&ver=1&signature=xNq6M1LNqUqHSjEY9j6kFKgsk9J5arq-dN9C9hzIeycyuRwNaC9sgrOWWIJBkvGeldBrf9Iwcj7CvqlMauwrXOVqFk1z-*IMckIe*dpvmHnTO*QFMXbCj8RzZTTNFzuNcX*yzYze6ma-NNJ4qeDe32C3rZeahmm0lW0YoG9kOsQ=
Trying Count: 1
请求成功！
新债王1月电话会
Crawling: http://mp.weixin.qq.com/s?timestamp=1561866012&src=3&ver=1&signature=xNq6M1LNqUqHSjEY9j6kFKgsk9J5arq-dN9C9hzIeycyuRwNaC9sgrOWWIJBkvGeldBrf9Iwcj7CvqlMauwrXB1hCL-5rkPZ3eQBr-VpPG1QLhbesaV99slVbkXb3cOXGPyLvDsdmBJS*LXEceqXHBwje35Q*fUT6U1BNOsEQZE=
Trying Count: 1
请求成功！
分享图片
Crawling: http://mp.weixin.qq.com/s?timestamp=1561866012&src=3&ver=1&signature=xNq6M1LNqUqHSjEY9j6kFKgsk9J5arq-dN9C9hzIeycyuRwNaC9sg




100%|██████████| 57/57 [12:06<00:00, 31.80s/it]

In [121]:
history

{'article': [{'abstract': '历史表明利率倒挂后美股却上涨，那么本轮会有何不同？',
   'author': '加州分析员',
   'content_url': 'http://mp.weixin.qq.com/s?timestamp=1561866012&src=3&ver=1&signature=xNq6M1LNqUqHSjEY9j6kFKgsk9J5arq-dN9C9hzIeycyuRwNaC9sgrOWWIJBkvGeldBrf9Iwcj7CvqlMauwrXCiEi-*yr9WDVXuWf6b5li911a-PkfKpoAtysAK9pB6mQPzD0ZM6fQTaDr5yR*2Jw2Z7efFm1FESvFOKa3-2pHY=',
   'copyright_stat': 11,
   'cover': 'http://mmbiz.qpic.cn/mmbiz_jpg/Bp4j7Ly7UpToAh7oLE71HkSKchjg3PZg8yibbAYWTMiaiadxEBzfz2iaCCcHl9vWQriawsibJQtpCOAmhiaxSIMIBIbeQ/0?wx_fmt=jpeg',
   'datetime': 1553473605,
   'fileid': 100000909,
   'main': 1,
   'send_id': 1000000113,
   'source_url': '',
   'title': '美股是否已经走到尽头？',
   'type': '49'},
  {'abstract': '市场观点：1) 10 yr 目前没有明显方向，trading sideway；2) 美股今年会破掉18年12月低位',
   'author': 'Jeff Gundlach',
   'content_url': 'http://mp.weixin.qq.com/s?timestamp=1561866012&src=3&ver=1&signature=xNq6M1LNqUqHSjEY9j6kFKgsk9J5arq-dN9C9hzIeycyuRwNaC9sgrOWWIJBkvGeldBrf9Iwcj7CvqlMauwrXOVqFk1z-*IMckIe*dpvmHnTO*QFMXbCj8RzZTTNFzuNcX

In [113]:
parse_detail(article)

{'content': '旧文重发。从7月的低点到现在，美国10年国债大跌，收益率上升了27个基点。未来一段时间，所有长久期资产很可能都会受到影响。这种趋势最终会传导到中国。而房子是典型的超长久期，流动性极差的资产。在这个时点加杠杆持有房地产非常不明智的。\n\n\n\n\n\n\n最新一期的GMO Quarterly Letter值得一看。管理一千多亿美元的GMO坚信价值投资，但同时也运用很多量化的方式分析市场，预测各大类资产的未来长期回报率，帮助进行资产配置。他们的一个核心理念就是长期看，各种资产具有均值回归的特性，而泡沫则是投资中最重要的事，最高的风险，最佳的机会，往往蕴含其中。\nGMO这封信的核心观点在于久期。过去6-7年，无论是股票还是债券，房地产还是PE股权投资，高风险资产还是低风险资产，流动性资产还是无流动性资产，表现的都非常好。他们之间的区别并不大。但是，最大的区别在于不同的久期。由于全球宽松，折现率一路下降，长久期的资产表现远远好过短久期的资产。因此，投资者在资产配置上也加大了长久期资产的配置，减少了短久期资产。但是，这种水涨船高的情况让长久期资产享受了太多超出历史均值的升值，未来很可能会均值回归，表现不佳。相反，未来短久期的“另类资产”很可能表现均值回归，超过长久期的资产。\n\n\n\n\n\n\n折现率下降对于长久期资产的价格提高具有极高的作用。举个例子，一个零票息的债券，不支付利息，只是在10年后一次性支付一百万美元的总额。这种债券的未来现金流是确定的，就是10年后支付一百万美元。如果是1.625%的收益率，则这个债券折现后的现值应该是$851,127美元。但是，如果收益率降到0.625%，则这个债券折现后的现值就是$939,596。这样一来，未来十年的收益，通过降低折现率，一下子就透支到了现在，实现了本来10年才能实现的收益的一半。而这个债券的现金流并没有任何变化，还是10年后支付一百万美元，所不同的是宽松的环境下，折现率降低，提前透支了长久期资产的价值。这种水涨船高，实际上是央行主导下的寅吃卯粮游戏。\n无论是长期债券、股票、房地产，还是需要长久期的PE股权投资，都受益于这种提前透支的游戏，水涨船高。但是那些短久期的资产，如现金、短期债券、对冲基金等则无法从折现率的下降中获得好处。\n未来，如果经济回升，长久期资产未来的现金流提高，但

In [ ]:
list(history)

In [ ]:
history['gzh']